<a href="https://colab.research.google.com/github/arminarj/convex-optimization-boyd/blob/master/Optimal_material_blending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install cvxpy==1.1.0a4 -q
# !pip install diffcp -q

     |████████████████████████████████| 993kB 3.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.2MB 3.3MB/s 


In [0]:
import numpy as np

m = 4 # the number of raw materials
n = 2 # the number of blended materials
q = 3 # the number of constituents

# the ith column of C is c_i
C = np.array([[ .9,  .8, .7, .6],
              [.08, .12, .2, .2],
              [.02, .08, .1, .2]])

# bounds on the blended product concentration
c_min = np.array([[.85, 0.65],
                    [  0,  0],
                    [  0,  0]])
c_max = np.array([[ 1,  1 ],
                  [.1, .18],
                  [.05, .17]])

FTilde = np.array([10, 10]) # limit on the flow rate of the blended material
F = np.array([7, 2, 6, 3]) # availibility of raw materials

p = np.array([15, 13, 11, 8]) # price of raw materials
pTilde = np.array([21, 18]) # price of the blended material




In [0]:
import cvxpy as cp

f = cp.Variable((n, m))
fTilde = cp.Variable(n)
C_tilde = cp.Variable((q, n))

obj_function =  - cp.sum(p @ f.T) + fTilde.T @ pTilde

obj = cp.Maximize(obj_function)

print("objective is DCP:", obj.is_dcp())


constraints  = [
                C_tilde.T @ np.ones(C_tilde.shape) == 1,
                c_min <= C_tilde,
                C_tilde <= c_max,
                fTilde <= FTilde,
                fTilde >= 0, 
                cp.sum(f, axis=0) <= F.T,
                cp.sum(f @ C.T , axis=0) == C_tilde @ fTilde,
]
for i in range(n):
    for j in range(m):
        constraints.append(f[i, j] >= 0)

prob = cp.Problem(obj, constraints)

print("prob is DCP:", prob.is_dcp())

try:
    prob.solve()
    print("status:  ", prob.status)
    print("optimal value:  ", prob.value)
except Exception as e:
    print(e)


objective is DCP: True
prob is DCP: False
Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
Sum(var34 @ [[0.9  0.08 0.02]
 [0.8  0.12 0.08]
 [0.7  0.2  0.1 ]
 [0.6  0.2  0.2 ]], 0, False) == var36 @ var35 , because the following subexpressions are not:
|--  var36 @ var35
